# Mirascope Quickstart Guide

1. [Setup](#Setup)
2. [Prompt Templates](#Prompt-Templates)
3. [Basic LLM Call](#Basic-LLM-Call)
4. [Streaming Responses](#Streaming-Responses)
5. [Asynchronous Processing](#Asynchronous-Processing)
6. [JSON Mode](#JSON-Mode)
7. [Response Models](#Response-Models)
8. [Output Parsers](#Output-Parsers)

## Setup

Mirascope supports various LLM providers, including [OpenAI](https://openai.com/), [Anthropic](https://www.anthropic.com/), [Mistral](https://mistral.ai/), [Gemini](https://gemini.google.com), [Groq](https://groq.com/), [Cohere](https://cohere.com/), [LiteLLM](https://www.litellm.ai/), [Azure AI](https://azure.microsoft.com/en-us/solutions/ai), and [Vertex AI](https://cloud.google.com/vertex-ai). For the purposes of this guide, we will be using OpenAI. Let's start by installing Mirascope and its dependencies:

In [ ]:
!pip install "mirascope[openai]"

This command installs Mirascope along with the necessary packages for the OpenAI integration.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
# Set the appropriate API key for the provider you're using

## Prompt Templates

Prompt templates in Mirascope allow you to create dynamic and reusable prompts.

In [10]:
from mirascope.core import prompt_template


@prompt_template("What is the capital of {country}?")
def get_capital_prompt(country: str): ...


print(get_capital_prompt("Japan"))

None


In this example:
1. The `@prompt_template` decorator defines the structure of the prompt.
2. `{country}` is a placeholder that gets replaced with the matching argument passed to the function.

Here's an example of a multi-line prompt:

In [11]:
@prompt_template(
    """
    SYSTEM: You are a helpful assistant that provides information about {topic}.
    USER: What are the key points to understand about {subtopic} in {topic}?
    """
)
def get_info_prompt(topic: str, subtopic: str): ...


print(get_info_prompt("artificial intelligence", "machine learning"))

None


This example:
1. Includes "SYSTEM" and "USER" keywords to define messages with different roles in the prompt.
2. Uses multiple variables (`{topic}` and `{subtopic}`) to create a more specific and dynamic prompt.

Prompt templates improve code readability, maintain consistency in prompts, and make it easier to experiment with and modify prompts for effective prompt engineering.

For more detailed information on prompt templates, check out our [Learn documentation on Prompts](https://docs.mirascope.io/learn/prompts).

## Basic LLM Call

The `call` decorator transforms functions with prompt templates into LLM API calls.

In [12]:
from mirascope.core import openai

# import provider-specific modules to use those providers (e.g. anthropic, gemini, etc.)


@openai.call("gpt-4o-mini")
@prompt_template("What is the capital of {country}?")
def get_capital(country: str): ...


response = get_capital("Japan")
print(response.content)

The capital of Japan is Tokyo.


For more advanced usage of LLM calls, refer to our [Learn documentation on Calls](https://docs.mirascope.io/learn/calls).

## Streaming Responses

Streaming allows you to process LLM responses in real-time.

In [13]:
@openai.call("gpt-4o-mini", stream=True)
@prompt_template("Provide a brief description of {city}.")
def stream_city_info(city: str): ...


for chunk, _ in stream_city_info("Tokyo"):
    print(chunk.content, end="", flush=True)

Tokyo, the capital city of Japan, is a vibrant metropolis known for its blend of traditional culture and cutting-edge modernity. With a population of over 13 million in the city proper and around 37 million in the Greater Tokyo Area, it is one of the most populous urban agglomerations in the world. Tokyo features a diverse range of neighborhoods, from the historic Asakusa with its ancient temples to the bustling shopping districts of Shibuya and Shinjuku.

The city is renowned for its unique architecture, from contemporary skyscrapers to historic shrines and gardens. Tokyo offers a rich tapestry of cuisine, shopping, and entertainment, including world-class dining, bustling markets, and numerous parks. Its efficient public transportation system makes it easy to explore various attractions, such as the iconic Tokyo Tower, the peaceful Meiji Shrine, and the lively Akihabara district known for its otaku culture.

Tokyo is also a major economic and cultural hub, hosting numerous internatio

For more information on streaming, including advanced techniques for processing streamed content, see our [Learn documentation on Streams](https://docs.mirascope.io/learn/streams).

## Asynchronous Processing

Mirascope supports asynchronous processing for efficient parallel execution of multiple LLM calls.

In [14]:
import asyncio


@openai.call("gpt-4o-mini")
@prompt_template("What is the capital of {country}?")
async def get_capital_async(country: str): ...


async def main():
    countries = ["France", "Japan", "Brazil"]
    tasks = [get_capital_async(country) for country in countries]
    results = await asyncio.gather(*tasks)
    for country, result in zip(countries, results, strict=False):
        print(f"The capital of {country} is {result.content}")


await main()

The capital of France is The capital of France is Paris.
The capital of Japan is The capital of Japan is Tokyo.
The capital of Brazil is The capital of Brazil is Brasília.


For more advanced usage of asynchronous processing, including async streaming, see our [Learn documentation on Async](https://docs.mirascope.io/learn/async).

## JSON Mode

JSON mode allows you to directly parse LLM outputs as JSON.

In [15]:
@openai.call("gpt-4o-mini", json_mode=True)
@prompt_template("Provide information about {city} in JSON format")
def city_info(city: str): ...


response = city_info("Tokyo")
print(response.content)  # This will be a JSON-formatted string

{
  "city": "Tokyo",
  "country": "Japan",
  "population": 13929286,
  "area_km2": 2191,
  "landmarks": [
    {
      "name": "Tokyo Tower",
      "type": "Tower",
      "height_m": 333
    },
    {
      "name": "Senso-ji Temple",
      "type": "Temple",
      "location": "Asakusa"
    },
    {
      "name": "Shibuya Crossing",
      "type": "Intersection",
      "description": "Famous pedestrian scramble"
    }
  ],
  "transportation": {
    "subway_system": true,
    "major_airports": [
      {
        "name": "Narita International Airport",
        "IATA_code": "NRT"
      },
      {
        "name": "Haneda Airport",
        "IATA_code": "HND"
      }
    ]
  },
  "culture": {
    "cuisine": [
      "Sushi",
      "Ramen",
      "Tempura"
    ],
    "traditional_events": [
      "Cherry Blossom Festival",
      "Gion Matsuri"
    ]
  }
}


Note that not all providers have an explicit JSON mode. For those providers, we attempt to instruct the model to provide JSON; however, there is no guarantee that it will output only JSON (it may start with some text like "Here is the JSON: ..."). This is where Output Parsers can be useful.

For more information on JSON mode, refer to our [Learn documentation on JSON Mode](https://docs.mirascope.io/learn/json-mode).

## Response Models

Response models allow you to structure and validate the output from LLMs.

In [16]:
from pydantic import BaseModel


class Capital(BaseModel):
    city: str
    country: str


@openai.call("gpt-4o-mini", response_model=Capital)
@prompt_template("Provide the capital of {country}")
def get_capital_info(country: str): ...


result = get_capital_info("Japan")
print(f"The capital of {result.country} is {result.city}")

The capital of Japan is Tokyo


For more details on response models, including advanced validation techniques, check our [Learn documentation on Response Models](https://docs.mirascope.io/learn/response-models).

## Output Parsers

Output parsers allow you to process LLM responses in custom formats.

In [17]:
class CountryInfo(BaseModel):
    country: str
    capital: str


def parse_country_and_capital(response: openai.OpenAICallResponse) -> CountryInfo:
    country, capital = response.content.split(" -> ")
    return CountryInfo(country=country, capital=capital)


@openai.call("gpt-4o-mini", output_parser=parse_country_and_capital)
@prompt_template(
    "What is the capital of {country}? Your output MUST follow the format 'country -> capital'."
)
def country_and_capital(country: str): ...


country_info = country_and_capital("Australia")
print(f"{country_info=}")
print(f"The capital of {country_info.country} is {country_info.capital}")

country_info=CountryInfo(country='Australia', capital='Canberra')
The capital of Australia is Canberra


For more information on output parsers, see our [Learn documentation on Output Parsers](https://docs.mirascope.io/learn/output-parsers).

This concludes our Quickstart Guide to Mirascope. We've covered the main features of the library, including prompt templates, basic calls, streaming, asynchronous processing, JSON mode, response models, and output parsers. Each of these features can be combined and customized to create powerful, flexible AI applications.

For more detailed information on each of these topics and advanced usage, including dynamic configuration and chaining, please refer to our comprehensive [Learn documentation](https://docs.mirascope.io/learn).